In [1]:
# !pip install altair
# !pip install altair_saver --upgrade
# !npm install -g vega-lite vega-cli canvas
# !pip install vl-convert-python --upgrade

In [2]:
# !pip install selenium --upgrade
# !apt-get install chromium-chromedriver -y

In [3]:
# !apt update
# !apt install ttf-mscorefonts-installer -y
# !apt reinstall fontconfig fontconfig-config libfontconfig1 -y

In [4]:
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Black.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Bold.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Book.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Medium.otf -P /usr/local/share/fonts

In [5]:
# !fc-cache -f

In [6]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [7]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [311]:
SAVE = True
LOCAL = False
DARK = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [312]:
%%capture pwd
!pwd

In [313]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
if not LOCAL:
    eco_git_home = (
        "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
    )
    vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
    colors = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
    )
    category_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
    )
    hue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
    )
    mhue_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
    )
    div_color = json.loads(
        requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
    )
    config = json.loads(
        requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
    )
else:
    eco_git_home = '/'.join(pwd.stdout.split("/")[:-2])+'/'
    vega_embed = open(eco_git_home + "guidelines/html/vega-embed.html",'r').read()
    colors = json.load(
        open(eco_git_home + "guidelines/colors/eco-colors.json",'r')
    )
    category_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-category-color.json",'r')
    )
    hue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-single-hue-color.json",'r')
    )
    mhue_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-multi-hue-color.json",'r')
    )
    div_color = json.load(
        open(eco_git_home + "guidelines/colors/eco-diverging-color.json",'r')
    )
    config = json.load(
        open(eco_git_home + "guidelines/charts/eco-global-config.json",'r')
    )
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
mo=0.5
height = config["height"]
width = config["width"]
uid, height, width

('how-did-uk-small-and-medium-sized-enterprises-manage-during-the-pandemic',
 300,
 500)

In [314]:
def save(df, f, LOCAL):
    fc = eco_git_path + f + ".csv"
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH", fc.replace("/data/", "/visualisation/").replace(".csv", ".json")
        )
    )
    if LOCAL:
        fc = df
    
    from IPython.display import display, HTML
    display(HTML(df.head().to_html()))
    
    readme = "## Figure " + f.replace('fig','').split('_')[0] + \
        '  \n\nData: [`csv`](data/' + f + '.csv)' +\
        '  \nGitHub: [' + f + '](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/'+uid +')'+\
        ''+\
        '  \n\n### Light theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local.png) [`svg`](visualisation/' + f + '_local.svg) [`json`](visualisation/' + f + '_local.json) '+ \
        '  \n (**Default**) Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '.png) [`svg`](visualisation/' + f + '.svg) [`json`](visualisation/' + f + '.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding.png) [`svg`](visualisation/' + f + '_local_no_branding.svg) [`json`](visualisation/' + f + '_local_no_branding.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding.png) [`svg`](visualisation/' + f + '_no_branding.svg) [`json`](visualisation/' + f + '_no_branding.json) '+ \
        ''+\
        '  \n\n### Dark theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_dark.png) [`svg`](visualisation/' + f + '_local_dark.svg) [`json`](visualisation/' + f + '_local_dark.json) '+ \
        '  \n Versions with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_dark.png) [`svg`](visualisation/' + f + '_dark.svg) [`json`](visualisation/' + f + '_dark.json)'+ \
        '  \nVersions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_no_branding_dark.png) [`svg`](visualisation/' + f + '_local_no_branding_dark.svg) [`json`](visualisation/' + f + '_local_no_branding_dark.json) '+ \
        '  \nVersions (no ECO branding) with data loaded from `GitHub`: ' + \
        '[`png`](visualisation/' + f + '_no_branding_dark.png) [`svg`](visualisation/' + f + '_no_branding_dark.svg) [`json`](visualisation/' + f + '_no_branding_dark.json) '+ \
        ''+\
        '  \n\n!["' + f + '"](visualisation/' + f + '.svg "' + f + '")\n\n' +\
        '  \n\n!["' + f + '_dark"](visualisation/' + f + '_dark.svg "' + f + '")\n\n' 
    return readme, f, fc

In [315]:
def area(base,color,opacity=1):
    return base.mark_area(opacity=opacity,
    interpolate="monotone",
    line={'color':color},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='#ffffff00', offset=0.2),
               alt.GradientStop(color=color, offset=0.8)],
        x1=1, #0.8
        y1=1,
        x2=1,
        y2=0
        )
    )

In [316]:
# service_color='#d6c8da' '#e4bfe2' '#ce4b96' colors['eco-turquiose']
service_color='#b4c8d8'
def dark(f):
    configSource = "visualisation/" + f + ".json"
    config = json.loads(open(configSource, "r").read())
    config['background']=colors['eco-background']
    service_color='#b4c8d8'
    for i in config['layer']:
        if 'encoding' in i:
            for x in ['x','y']:
                if x in i['encoding']:
                    if 'axis' in i['encoding'][x]:
                        for c in ['domainColor','labelColor','tickColor','titleColor','gridColor']:
                            if c in i['encoding'][x]['axis']:
                                i['encoding'][x]['axis'][c]=service_color
        if 'mark' in i:
            if 'color' in i['mark']:
                if i['mark']['color']==colors['eco-gray']:
                    i['mark']['color']=service_color
                elif i['mark']['color']==colors['eco-blue']:
                    i['mark']['color']=colors['eco-yellow']
                elif i['mark']['color']==service_color:
                    i['mark']['color']=colors['eco-green']
                elif 'stops' in i['mark']['color']:
                    for s in i['mark']['color']['stops']:
                        if 'color' in s:
                            if s['color']==colors['eco-gray']:
                                s['color']=service_color
                            elif s['color']==colors['eco-blue']:
                                s['color']=colors['eco-yellow']
                            elif s['color']==service_color:
                                s['color']=colors['eco-green']
            if 'line' in i['mark']:
                if 'color' in i['mark']['line']:
                    if i['mark']['line']['color']==colors['eco-gray']:
                        i['mark']['line']['color']=service_color
                    elif i['mark']['line']['color']==colors['eco-blue']:
                        i['mark']['line']['color']=colors['eco-yellow']
                    elif i['mark']['line']['color']==service_color:
                        i['mark']['line']['color']=colors['eco-green']
    if 'datasets' in config:
        for i in config['datasets']:
            if 'img' in config['datasets'][i][0]:
                if 'eco-icon-dark' in config['datasets'][i][0]['img']:
                    config['datasets'][i][0]['img']=config['datasets'][i][0]['img'].replace('eco-icon-dark','eco-icon-light')
    return alt.Chart.from_dict(config) 

# Fig 1

In [317]:
df = pd.read_csv("raw/d1.csv")

In [318]:
readme, f, fc = save(df,"fig1_loan",LOCAL)

,Region,Country,y1,y2,y3
0,Yorkshire and the Humber,England,7.141404,11.094989,8.586595
1,West Midlands,England,8.056968,11.182783,8.749799
2,Wales,Wales,3.483394,10.507524,7.897478
3,South West,England,9.380015,11.332137,8.915261
4,South East,England,15.573199,11.956020,9.411207


In [319]:
order=df.sort_values(by='Region').sort_values(by='Country')['Region'].values

In [320]:
xmin=0
xmax=20
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':'London'},{'x':xmax,'y':'London'}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=0).encode(
    alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:N",
        sort=[],
        axis=alt.Axis(
            grid=True,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            tickOffset=18,
            title="",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelOffset=-18,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(y=alt.Y('Region:N',scale=alt.Scale(domain=order)))
line1=base.mark_bar(color=colors['eco-dot'],height=10,opacity=0.8).encode(x='y1:Q')
line2=base.mark_bar(color=colors['eco-light-blue'],height=10,yOffset=-10,opacity=0.8).encode(x='y2:Q')
line3=base.mark_bar(color=colors['eco-turquiose'],height=10,yOffset=10,opacity=0.8).encode(x='y3:Q')
label1=line1.mark_text(text='Businesses',color=colors['eco-dot'],align='left',dx=5,dy=4,size=13).transform_filter('datum.Region=="North West"')
label2=line2.mark_text(text='Predicted BBL',color=colors['eco-light-blue'],align='left',dx=5,dy=-8,size=13).transform_filter('datum.Region=="North East"')
label3=line3.mark_text(text='Predicted CBILS',color=colors['eco-turquiose'],align='left',dx=5,dy=11,size=13).transform_filter('datum.Region=="North East"')

layer1 = (
    (xaxis+line1+line2+line3+label1+label2+label3).properties(height=450, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

if DARK:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.json")
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 'East Midlands', "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-3,opacity=mo,xOffset=0).encode(x='x:Q',y='y:N',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

if DARK:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "w").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig1_loan



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 2

In [321]:
df = pd.read_csv("raw/d2.csv")
df.columns=['x1','y1','x2','y2','x3','y3']

In [322]:
readme, f, fc = save(df,"fig2_infl",LOCAL)

,x1,y1,x2,y2,x3,y3
0,2.118391,0.006108,0.978071,-0.000345,1.206135,0.012724
1,2.603027,0.018647,1.605247,-0.000107,1.833311,0.008970
2,3.144679,0.029262,2.232423,-0.000107,2.460487,0.007116
3,3.771855,0.031923,2.859599,-0.000107,3.087663,0.006689
4,4.399031,0.034870,3.486775,-0.000107,3.714839,0.006689


In [323]:
xmin=1
xmax=28
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="months since March 2020",
            titleY=-15,
            titleX=328,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(domain=[xmin, xmax])
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="%",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.1f',
            tickCount=8
        ),
        scale=alt.Scale(domain=[0, 0.2]),
    )
)

base = alt.Chart(fc)
line1=base.mark_line(color=colors['eco-turquiose'],size=3,opacity=0.8).encode(x='x1:Q',y='y1:Q')
line2=base.mark_line(color=colors['eco-light-blue'],size=3,opacity=0.8).encode(x='x2:Q',y='y2:Q')
line3=base.mark_line(color=colors['eco-dot'],size=3,opacity=0.8).encode(x='x3:Q',y='y3:Q')
label1=line1.mark_text(text='Repaid CBIL',color=colors['eco-turquiose'],align='left',dx=5,size=13).transform_filter('datum.x1>27.8')
label2=line2.mark_text(text='Repaid BBL',color=colors['eco-light-blue'],align='left',dx=5,size=13).transform_filter('datum.x2>27.6')
label3=line3.mark_text(text='Inflation',color=colors['eco-dot'],align='left',dx=5,size=13).transform_filter('datum.x3>27.6')

layer1 = (
    (xaxis+line1+line2+line3+label1+label2+label3).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 0.2, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=-0).encode(x='x:Q',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

fig2_infl



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 3

In [324]:
df=pd.DataFrame([{'q':'Q1','v':7.6,'y':2021},{'q':'Q1','v':4.9,'y':2022},{'q':'Q2','v':5.2,'y':2021},
                 {'q':'Q3','v':4.9,'y':2021},{'q':'Q4','v':4.8,'y':2021}])

In [325]:
readme, f, fc = save(df,"fig3_lending",LOCAL)

,q,v,y
0,Q1,7.6,2021
1,Q1,4.9,2022
2,Q2,5.2,2021
3,Q3,4.9,2021
4,Q4,4.8,2021


In [326]:
xmin='Q1'
xmax='Q4'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="£ billion",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(x='q:N')
line1=base.mark_bar(color=colors['eco-turquiose'],opacity=0.8,width=36,xOffset=-20).encode(y='v:Q').transform_filter('datum.y==2021')
line2=base.mark_bar(color=colors['eco-light-blue'],opacity=0.8,width=36,xOffset=20).encode(y='v:Q').transform_filter('datum.y==2022')
labels1=line1.mark_text(color='#ffffff',width=36,xOffset=-20,yOffset=12,size=13).encode(y='v:Q',text='v:N')
labels2=line2.mark_text(color='#ffffff',width=36,xOffset=20,yOffset=12,size=13).encode(y='v:Q',text='v:N')
labels3=line1.mark_text(color=colors['eco-turquiose'],width=36,xOffset=20,yOffset=12,size=13).encode(y='v:Q',text='y:N').transform_filter('datum.q=="Q1"')
labels4=line2.mark_text(color=colors['eco-light-blue'],width=36,xOffset=20,yOffset=-12,size=13).encode(y='v:Q',text='y:N').transform_filter('datum.q=="Q1"')

layer1 = (
    (xaxis+line1+line2+labels1+labels2+labels3+labels4).properties(height=300, width=400, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": xmax, "y": 8, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=130).encode(x='x:N',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.


fig3_lending



alt.LayerChart(...)

alt.LayerChart(...)

# Fig 4

In [327]:
df = pd.read_csv("raw/d4.csv")
df.columns=['a','b','c']
df['d1']=df['c']-df['b']
df['d2']=df['b']-df['a']
df['c']=df['d1']
df['b']=df['d2']
df=df.drop(['d1','d2'],axis=1)
df.columns=['Seed','Early stage venture',' Late stage venture']
df=df.stack().reset_index()
df.columns=['a','b','c']
df['a']=[str(2007+i//4)+'-Q'+str(i%4+1) for i in df['a']]

In [328]:
readme, f, fc = save(df,"fig4_venture",LOCAL)

,a,b,c
0,2007-Q1,Seed,24.536729
1,2007-Q1,Early stage venture,35.127353
2,2007-Q1,Late stage venture,9.789590
3,2007-Q2,Seed,6.109265
4,2007-Q2,Early stage venture,17.851606


In [329]:
xmin='2007-Q1'
xmax='2020-Q1'
xaxis = alt.Chart(pd.DataFrame([{'x':xmin,'y':0},{'x':xmax,'y':0}])).mark_line(color=colors["eco-gray"],opacity=mo-0.2,strokeWidth=1).encode(
    alt.X(
        "x:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            orient="bottom",
            labelAngle=0,
            # labelExpr="indexof(datum.value,'Q1')>-1?slice(datum.value,0,4)+' '+slice(datum.value,4):slice(datum.value,4)"
            labelExpr="indexof(datum.value,'2007-Q1')>-1?slice(datum.value,0,4):indexof(datum.value,'2017-Q1')>-1?slice(datum.value,0,4):indexof(datum.value,'2020-Q1')>-1?slice(datum.value,0,4):indexof(datum.value,'2011-Q1')>-1?slice(datum.value,0,4):indexof(datum.value,'2014-Q1')>-1?slice(datum.value,0,4):indexof(datum.value,'2009-Q1')>-1?datum.value:''"
        ),
        # scale=alt.Scale(domain=[xmin, xmax])
    ),
    y=alt.Y(
        "y:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="£ million",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            format='.0f',
            tickCount=8
        ),
        # scale=alt.Scale(domain=[-0.16, 0.06]),
    )
)

base = alt.Chart(fc).encode(x=alt.X('a:N'))
bars=base.mark_bar(opacity=0.8,width=7).encode(y=alt.Y('c:Q'),color=alt.Color('b:N',scale=alt.Scale(range=[colors['eco-turquiose'],
    colors['eco-dot'],colors['eco-light-blue']]),legend=alt.Legend(title='',labelColor=colors["eco-gray"])))

layer1 = (
    (xaxis+bars).properties(height=300, width=500, title="")
    .configure(font='Circular Std Book').configure_view(stroke=None)
)

layer1.save("visualisation/" + f + "_no_branding.json")
if DARK:
    layer2=dark(f+'_no_branding')
    layer2.save("visualisation/" + f + "_no_branding_dark.json")
    
if SAVE:
    layer1.save("visualisation/" + f + "_no_branding.svg")
    layer1.save("visualisation/" + f + "_no_branding.png")
    if DARK:
        layer2.save("visualisation/" + f + "_no_branding_dark.svg")
        layer2.save("visualisation/" + f + "_no_branding_dark.png")
    
logo=alt.Chart(pd.DataFrame([{"x": '2020-Q1', "y": 500, "img": "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/logos/eco-icon-dark.png"}]))\
    .mark_image(width=40,height=40,align='right',baseline='top',yOffset=-33,opacity=mo,xOffset=-0).encode(x='x:N',y='y:Q',url='img:N')
# ecomark=alt.Chart(pd.DataFrame([{"x": '2019-01-01', "y": 14}]))\
#     .mark_point(size=100,fill=colors['eco-turquiose'],stroke=None,opacity=1,xOffset=-20,yOffset=-20).encode(x='x:T',y='y:Q')
# layer1+=(ecomark)
# layer1+=(logo)

layer1.save("visualisation/" + f + ".json")
if DARK:
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
if SAVE:
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    if DARK:
        layer2.save("visualisation/" + f + "_dark.svg")
        layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if DARK:
    layer2.display()

WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.


fig4_venture



alt.LayerChart(...)

alt.LayerChart(...)

# Post-process

## Clean up `SVG`s
`base64 URI` encode images

In [330]:
from base64 import b64encode

In [331]:
from os import listdir
from os.path import isfile, join
mypath='./visualisation/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
svgs=[i for i in onlyfiles if i[-4:]=='.svg']

In [332]:
for svg in svgs:
    s=open(mypath+svg,'r').read()
    si=s.find('"image mark" xlink:href=')
    if si>-1:
        s2=s[si+25:]
        imgurl=s2[:s2.find('"')]
        if 'data:image/png;' not in imgurl:
            s1=s[:si+25]+s2.replace(imgurl,'data:image/png;base64,'+b64encode(requests.get(imgurl).content).decode('utf-8'))
            open(mypath+svg,'w').write(s1)
            print(svg)

MissingSchema: Invalid URL '': No scheme supplied. Perhaps you meant https://?

## README

In [333]:
from IPython.display import display, Markdown

with open('README.md', 'r') as fh:
    content = fh.read()

display(Markdown(content))

## Figure 1  

Data: [`csv`](data/fig1_loan.csv)  
GitHub: [fig1_loan](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-did-uk-small-and-medium-sized-enterprises-manage-during-the-pandemic)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_loan_local.png) [`svg`](visualisation/fig1_loan_local.svg) [`json`](visualisation/fig1_loan_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_loan.png) [`svg`](visualisation/fig1_loan.svg) [`json`](visualisation/fig1_loan.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_loan_local_no_branding.png) [`svg`](visualisation/fig1_loan_local_no_branding.svg) [`json`](visualisation/fig1_loan_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_loan_no_branding.png) [`svg`](visualisation/fig1_loan_no_branding.svg) [`json`](visualisation/fig1_loan_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_loan_local_dark.png) [`svg`](visualisation/fig1_loan_local_dark.svg) [`json`](visualisation/fig1_loan_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig1_loan_dark.png) [`svg`](visualisation/fig1_loan_dark.svg) [`json`](visualisation/fig1_loan_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig1_loan_local_no_branding_dark.png) [`svg`](visualisation/fig1_loan_local_no_branding_dark.svg) [`json`](visualisation/fig1_loan_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig1_loan_no_branding_dark.png) [`svg`](visualisation/fig1_loan_no_branding_dark.svg) [`json`](visualisation/fig1_loan_no_branding_dark.json)   

!["fig1_loan"](visualisation/fig1_loan.svg "fig1_loan")

  

!["fig1_loan_dark"](visualisation/fig1_loan_dark.svg "fig1_loan")

## Figure 2  

Data: [`csv`](data/fig2_infl.csv)  
GitHub: [fig2_infl](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-did-uk-small-and-medium-sized-enterprises-manage-during-the-pandemic)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_infl_local.png) [`svg`](visualisation/fig2_infl_local.svg) [`json`](visualisation/fig2_infl_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_infl.png) [`svg`](visualisation/fig2_infl.svg) [`json`](visualisation/fig2_infl.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_infl_local_no_branding.png) [`svg`](visualisation/fig2_infl_local_no_branding.svg) [`json`](visualisation/fig2_infl_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_infl_no_branding.png) [`svg`](visualisation/fig2_infl_no_branding.svg) [`json`](visualisation/fig2_infl_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_infl_local_dark.png) [`svg`](visualisation/fig2_infl_local_dark.svg) [`json`](visualisation/fig2_infl_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig2_infl_dark.png) [`svg`](visualisation/fig2_infl_dark.svg) [`json`](visualisation/fig2_infl_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig2_infl_local_no_branding_dark.png) [`svg`](visualisation/fig2_infl_local_no_branding_dark.svg) [`json`](visualisation/fig2_infl_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig2_infl_no_branding_dark.png) [`svg`](visualisation/fig2_infl_no_branding_dark.svg) [`json`](visualisation/fig2_infl_no_branding_dark.json)   

!["fig2_infl"](visualisation/fig2_infl.svg "fig2_infl")

  

!["fig2_infl_dark"](visualisation/fig2_infl_dark.svg "fig2_infl")

## Figure 3  

Data: [`csv`](data/fig3_lending.csv)  
GitHub: [fig3_lending](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-did-uk-small-and-medium-sized-enterprises-manage-during-the-pandemic)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_lending_local.png) [`svg`](visualisation/fig3_lending_local.svg) [`json`](visualisation/fig3_lending_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig3_lending.png) [`svg`](visualisation/fig3_lending.svg) [`json`](visualisation/fig3_lending.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_lending_local_no_branding.png) [`svg`](visualisation/fig3_lending_local_no_branding.svg) [`json`](visualisation/fig3_lending_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig3_lending_no_branding.png) [`svg`](visualisation/fig3_lending_no_branding.svg) [`json`](visualisation/fig3_lending_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_lending_local_dark.png) [`svg`](visualisation/fig3_lending_local_dark.svg) [`json`](visualisation/fig3_lending_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig3_lending_dark.png) [`svg`](visualisation/fig3_lending_dark.svg) [`json`](visualisation/fig3_lending_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig3_lending_local_no_branding_dark.png) [`svg`](visualisation/fig3_lending_local_no_branding_dark.svg) [`json`](visualisation/fig3_lending_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig3_lending_no_branding_dark.png) [`svg`](visualisation/fig3_lending_no_branding_dark.svg) [`json`](visualisation/fig3_lending_no_branding_dark.json)   

!["fig3_lending"](visualisation/fig3_lending.svg "fig3_lending")

  

!["fig3_lending_dark"](visualisation/fig3_lending_dark.svg "fig3_lending")

## Figure 4  

Data: [`csv`](data/fig4_venture.csv)  
GitHub: [fig4_venture](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/articles/how-did-uk-small-and-medium-sized-enterprises-manage-during-the-pandemic)  

### Light theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_venture_local.png) [`svg`](visualisation/fig4_venture_local.svg) [`json`](visualisation/fig4_venture_local.json)   
 (**Default**) Versions with data loaded from `GitHub`: [`png`](visualisation/fig4_venture.png) [`svg`](visualisation/fig4_venture.svg) [`json`](visualisation/fig4_venture.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_venture_local_no_branding.png) [`svg`](visualisation/fig4_venture_local_no_branding.svg) [`json`](visualisation/fig4_venture_local_no_branding.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig4_venture_no_branding.png) [`svg`](visualisation/fig4_venture_no_branding.svg) [`json`](visualisation/fig4_venture_no_branding.json)   

### Dark theme  

Versions with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_venture_local_dark.png) [`svg`](visualisation/fig4_venture_local_dark.svg) [`json`](visualisation/fig4_venture_local_dark.json)   
 Versions with data loaded from `GitHub`: [`png`](visualisation/fig4_venture_dark.png) [`svg`](visualisation/fig4_venture_dark.svg) [`json`](visualisation/fig4_venture_dark.json)  
Versions (no ECO branding) with data locally embedded into the `Vega-lite` specification file: [`png`](visualisation/fig4_venture_local_no_branding_dark.png) [`svg`](visualisation/fig4_venture_local_no_branding_dark.svg) [`json`](visualisation/fig4_venture_local_no_branding_dark.json)   
Versions (no ECO branding) with data loaded from `GitHub`: [`png`](visualisation/fig4_venture_no_branding_dark.png) [`svg`](visualisation/fig4_venture_no_branding_dark.svg) [`json`](visualisation/fig4_venture_no_branding_dark.json)   

!["fig4_venture"](visualisation/fig4_venture.svg "fig4_venture")

  

!["fig4_venture_dark"](visualisation/fig4_venture_dark.svg "fig4_venture")

